# Предобработка данных

В данном ноутбуке данные, полученные с сайта "musik-produktiv.com" будут приведены к общему виду, пригодному для анализа и будут заполнены пропуски. 

Особенности запонения пропусков и работы с данными в этой части проекта:
- В связи с тем, что некоторые пропущенные значения невозможно восстановить на основании соседних столбцов, они будут заполнены на основании личного опыта автора и знания рынка бас-гитар. 
- Данные с сайта зачастую не заполнены единообразно - зачастую разные производители инструментов называют один и те же материалы для своих инструментов и комплектующие по-разному из-за чего данные требуют масштабного препроцессинга, и зачастную данные необходимо обрабатывать вручную, так как нет возможности автоматизировать этот процесс, но там где это возможно - автоматизация будет применяться.

In [1]:
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('musik_productiv_df')

In [3]:
df.head()

,Наименование,Цена,Форма корпуса,Материал корпуса,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель
0,Höfner Ignition Club Bass VSB,25388,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,КНР,NaN,NaN,NaN,NaN
1,Höfner Violin Ignition Bass SB,24940,Битлз бас,Клён (мэйпл),Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,Соупбар,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,NaN,NaN
2,ESP LTD B-205 SM STBLKS,50432,"Форма ""Даблкатэвей""",Ясень,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,NaN
3,Ibanez Gio GSR200B WNF,18593,Саундджиэр,Нато (Ньято / Мора),NaN,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Ятоба,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,Индонезия,PSNDP,PSNDJ,NaN,NaN
4,Ibanez Artcore AGB200-NT,41742,Полуакустическая (Семи Холлоу Бади),Клён (мэйпл),Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3"" (770 мм)",Трёхслойный клён/махагони,Индийский лорел (терминалия крылатая),22,четырёхструнная,Хамбакер,КНР,Ибанез клэссик элит,Ибанез клэссик элит,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Наименование              600 non-null    object
 1   Цена                      600 non-null    int64 
 2   Форма корпуса             570 non-null    object
 3   Материал корпуса          579 non-null    object
 4   Дека                      170 non-null    object
 5   Конструкция               587 non-null    object
 6   Конструкция грифа         588 non-null    object
 7   Мензура                   586 non-null    object
 8   Длина мензуры             585 non-null    object
 9   Гриф                      585 non-null    object
 10  Накладка грифа            580 non-null    object
 11  Количество ладов          582 non-null    object
 12  Число струн               598 non-null    object
 13  Типы звукоснимателей      534 non-null    object
 14  Страна производства       

Разделим столбец "Наименование" на "Бренд" и "Модель".

In [5]:
brand = df['Наименование'].str.split(' ', expand = True)[0]
model = df['Наименование'].str.split(' ', expand = True)[1]

In [6]:
df['Бренд'] = brand
df['Модель'] = model
df = df.drop('Наименование', axis = 1)

In [7]:
df['Форма корпуса'].unique()

array(['Битлз бас', 'Форма "Даблкатэвей"', 'Саундджиэр',
       'Полуакустическая (Семи Холлоу Бади)',
       'Холлоубади, специальная полуакустическая конструкция', 'Дабл кат',
       'Басс Воркшоп', 'Пресижн бас', 'Феникс', nan,
       'Ноун Реверс Тандербёрд', 'Старфайр', 'Форма "Стинг рэй"',
       'ESP F-Форма', 'Дизайнер', 'Форма "LTD B"', 'ВТВ', 'Лес пол',
       'Тальман', 'Форма "Джаз бас"', 'КЗС (RGB)',
       'Сингл Кат (с одним вырезом)', 'G&amp;L Басс', 'Бонги',
       'Форма "Тандербирд"', 'Спектор', 'Арткор AF', 'Президент Басс',
       'Лес Пол Джуниор Дабл Кат', 'Форма "Рикенбекер"', 'Фоти Эйт',
       'Айсмэн', 'Эмбэсси', 'Ретро', 'Форма "Ворлок"', 'Мокингбёрд',
       'Видоу', 'Бензиновая канистра', 'Оффсет (сдвинутый)', 'Стрим',
       'EHB', 'Дестройер', '"Лонгхорн"'], dtype=object)

Варианты формы корпуса очень многообразы и зачастую не несут в себе дополнительного смысла - это скорее маркетинговый признак - эти данные дублируются в других столбцах - удалим признак "Форма корпуса".

In [8]:
df = df.drop('Форма корпуса', 1)

In [9]:
df['Материал корпуса'].value_counts()

Ольха                                                                                    132
Махагони                                                                                  90
Ясень                                                                                     76
Плакучий ясень                                                                            54
Окоуме                                                                                    47
Клён (мэйпл)                                                                              37
Американская липа                                                                         33
Тополь                                                                                    29
Нато (Ньято / Мора)                                                                       21
Ньято                                                                                     11
Ламинированный махагони                                               

Приведем признак "Материал корпуса" с единообразию и каждый из используемых материалов выведем в отдельный столбец.

In [10]:
df['Материал корпуса'] = df['Материал корпуса'].replace('Клён (мэйпл)', 'Клён')
df['Материал корпуса'] = df['Материал корпуса'].replace('Дерево-каури (агатис)', 'Агатис')
df['Материал корпуса'] = df['Материал корпуса'].replace('Плакучий ясень', 'Ясень')
df['Материал корпуса'] = df['Материал корпуса'].replace('Нато (Ньято / Мора)', 'Нато')
df['Материал корпуса'] = df['Материал корпуса'].replace('Ньято', 'Нато')
df['Материал корпуса'] = df['Материал корпуса'].replace('Африканский махагони', 'Махагони')
df['Материал корпуса'] = df['Материал корпуса'].replace('Махагони c кленовой сердцевиной', 'Махагони / Клён')
df['Материал корпуса'] = df['Материал корпуса'].replace('Ламинированный махагони', 'Махагони')
df['Материал корпуса'] = df['Материал корпуса'].replace('Многослойная конструкция: структурный клён, бубинга, ясень, бубинга, структурный клён', 'Клён / Бубинга / Ясень')
df['Материал корпуса'] = df['Материал корпуса'].replace('Трёхслойный венге / махагони / венге', 'Венге / Махагони')
df['Материал корпуса'] = df['Материал корпуса'].replace('Окоуме / ясень', 'Окоуме / Ясень')
df['Материал корпуса'] = df['Материал корпуса'].replace('Бубинга / ясень / бубинга', 'Бубинга / Ясень / Бубинга')
df['Материал корпуса'] = df['Материал корпуса'].replace('Махагони с внутренними звуковыми камерами', 'Махагони')
df['Материал корпуса'] = df['Материал корпуса'].replace('Австралийское чёрное дерево и задняя часть из клёна', 'Черное дерево / Клён')
df['Материал корпуса'] = df['Материал корпуса'].replace('Древесина ясеня, подвергнутая специальной термической обработке', 'Ясень')
df['Материал корпуса'] = df['Материал корпуса'].replace('Ясень / махагони', 'Ясень / Махагони')
df['Материал корпуса'] = df['Материал корпуса'].replace('Орех / клён', 'Орех / Клён')
df['Материал корпуса'] = df['Материал корпуса'].replace('Клён с резонаторными камерами', 'Клён')
df['Материал корпуса'] = df['Материал корпуса'].replace('Белая жесть с рамкой из древесины липы', 'Жесть, Липа')

In [11]:
df_body = df['Материал корпуса'].str.split(' / ', expand = True)
df_body.columns = ['Материал корпуса_1', 'Материал корпуса_2', 'Материал корпуса_3']
df = pd.concat([df, df_body], axis = 1)
df = df.drop('Материал корпуса', axis = 1)
df.rename(columns = {'Материал корпуса_1' : 'Материал корпуса'}, inplace = True)
df

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,...,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель,Бренд,Модель,Материал корпуса,Материал корпуса_2,Материал корпуса_3
0,25388,Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,...,КНР,NaN,NaN,NaN,NaN,Höfner,Ignition,Клён,None,None
1,24940,Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5"" (777 мм)",Трёхслойный клён/бук,Палисандр (розовое дерево),22,четырёхструнная,...,КНР,Хофнер Игнишн Стэйпл Никел,Хофнер Игнишн Стэйпл Никел,NaN,NaN,Höfner,Violin,Клён,None,None
2,50432,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Пятислойный клён/орех,Палисандр (розовое дерево),24,пятиструнная,...,Индонезия,ESP дизайнд SB-5N,ESP дизайнд SB-5B,Трёхполосный эквалайзер,NaN,ESP,LTD,Ясень,None,None
3,18593,NaN,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Ятоба,22,четырёхструнная,...,Индонезия,PSNDP,PSNDJ,NaN,NaN,Ibanez,Gio,Нато,None,None
4,41742,Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3"" (770 мм)",Трёхслойный клён/махагони,Индийский лорел (терминалия крылатая),22,четырёхструнная,...,КНР,Ибанез клэссик элит,Ибанез клэссик элит,NaN,NaN,Ibanez,Artcore,Клён,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,88037,NaN,Солид Бади,Закреплён винтами,Сверхдлинная,"35"" (889 мм)","3-слойный, выполненный из специально термическ...",Чёрное дерево (Эбони),24,пятиструнная,...,Южная Корея,NaN,EMG J,Двухполосный эквалайзер,EMG P,Spector,NS,Ясень,None,None
596,30541,NaN,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,...,КНР,NaN,Спектор Джи,NaN,Спектор P,Spector,Performer,Нато,None,None
597,32781,NaN,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Трёхслойный клён,Амарант,24,четырёхструнная,...,КНР,NaN,Спектор Джи,NaN,Спектор P,Spector,Performer,Нато,None,None
598,51823,NaN,Солид Бади,Закреплён винтами,Длинная,"34"" (864 мм)",Клён (мэйпл),Клён (мэйпл),21,четырёхструнная,...,Индонезия,NaN,Керамический Хамбакер,Двухполосный эквалайзер,NaN,Sterling,by,Махагони,None,None


In [12]:
df['Материал корпуса_2'] = df['Материал корпуса_2'].fillna('Нет')
df['Материал корпуса_3'] = df['Материал корпуса_3'].fillna('Нет')

Посмотрим у каких брендов есть незаполненные значения столбца "Материал корпуса".

In [13]:
df[df['Материал корпуса'].isna()]['Бренд'].unique()

array(['Ibanez', 'Höfner', 'Sandberg', 'Schecter', 'Danelectro'],
      dtype=object)

Напишем функцию, которая будет принимать на вход 2 признака: 'column', 'col2', группировать данные по 'col2', и заполнять пропущенные значения 'column' самыми популярными значениями в рамках группы.
Например: заполним пропущенные значение материалов корпуса самыми популярными значениями того или иного бренда.

In [14]:
def fill(column, col2):
    for i in list(df[df[column].isna()][col2].unique()):
        df_filtered = df[df[col2] == i]
        return(pd.Series(df[column].fillna(df_filtered[column].mode()[0])))

In [15]:
df['Материал корпуса'] = fill('Материал корпуса', 'Бренд')

In [16]:
df['Дека'].value_counts()

Тополь                                                                 35
Клён (мэйпл)                                                           17
Клён, со структурой языков пламени (пламя / тигр / волнистый мэйпл)    16
Клён, расщеплённый (неповторимый эффект)                               12
Фигурный структурный клён                                               9
Ясень                                                                   8
Ель                                                                     7
Флэймд Мэйпл                                                            6
Ламинированный махагони                                                 5
Махагони, выпуклая форма                                                4
Структурный клён (Квилт Мэйпл)                                          4
Древесина орешины                                                       3
Бокоте                                                                  3
Венге                                 

Столбец "Дека" заполнен вариативными значениями, однако лучше сохранить эту вариантивность, так как возможно, что структурные особенности одного и того же материала имеют влияние на цену.
Также заполним столбец "Дека" самыми популярными значениями в зависимости от бренда.

In [17]:
df['Дека'] = fill('Дека', 'Бренд')

In [18]:
df['Конструкция'].value_counts()

Солид Бади                             552
Полуакустическая (Семи Холлоу Бади)     29
Полуакустика (Холлоу Бади)               5
Семи-полуакустика                        1
Name: Конструкция, dtype: int64

заполним столбец "Конструкция" самыми популярными значениями в зависимости от бренда.

In [19]:
df['Конструкция'] = fill('Конструкция', 'Бренд')

'Полуакустика (Холлоу Бади)' - то же самое, что 'Полуакустическая (Семи Холлоу Бади)'.

In [20]:
df['Конструкция'] = df['Конструкция'].replace('Полуакустика (Холлоу Бади)', 'Полуакустическая (Семи Холлоу Бади)')

In [21]:
df['Конструкция'].value_counts()

Солид Бади                             565
Полуакустическая (Семи Холлоу Бади)     34
Семи-полуакустика                        1
Name: Конструкция, dtype: int64

In [22]:
df['Конструкция грифа'].value_counts()

Закреплён винтами       468
Сплошной с декой         66
Вклеен в корпус          51
Склеен по всей длине      2
Полированный чёрный       1
Name: Конструкция грифа, dtype: int64

In [23]:
df[df['Конструкция грифа'] == 'Склеен по всей длине']

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,...,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель,Бренд,Модель,Материал корпуса,Материал корпуса_2,Материал корпуса_3
285,129594,"Клён, со структурой языков пламени (пламя / ти...",Солид Бади,Склеен по всей длине,Сверхдлинная,"35"" (889 мм)",Пятислойный из древесины клёна / пёрплхарт (ам...,Макассар Эбони,24,пятиструнная,...,NaN,Фишмэн Флюэнс SB-1,Фишмэн Флюэнс SB-1,Двухполосный эквалайзер,NaN,ESP,LTD,Махагони,Нет,Нет
385,302416,Ель,Полуакустическая (Семи Холлоу Бади),Склеен по всей длине,Короткая,"30"" (762 мм)",Трёхслойный клён,Розовое дерево/палисандр,21,четырёхструнная,...,NaN,Хофнер Блэк Бар H517,Хофнер Блэк Бар H517,NaN,NaN,Höfner,H500/5,Клён,Нет,Нет


In [24]:
df[df['Конструкция грифа'] == 'Полированный чёрный']

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,...,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель,Бренд,Модель,Материал корпуса,Материал корпуса_2,Материал корпуса_3
40,141800,Тополь,Солид Бади,Полированный чёрный,Длинная,"34"" (864 мм)",Клён (мэйпл),Чёрное дерево (Эбони),22,четырёхструнная,...,Германия,Зaндберг,Зaндберг,Двухполосный эквалайзер,NaN,Sandberg,California,Ольха,Нет,Нет


Можно заменить значения 'Склеен по всей длине' на 'Вклеен в корпус' и 'Полированный чёрный' на 'Закреплён винтами'.

In [25]:
df['Конструкция грифа'] = df['Конструкция грифа'].replace('Склеен по всей длине', 'Вклеен в корпус')
df['Конструкция грифа'] = df['Конструкция грифа'].replace('Полированный чёрный', 'Закреплён винтами')

In [26]:
df[df['Конструкция грифа'].isna()]

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,...,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель,Бренд,Модель,Материал корпуса,Материал корпуса_2,Материал корпуса_3
275,101553,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Sandberg,Classic,Махагони,Нет,Нет
315,55929,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Omen,Махагони,Нет,Нет
561,89530,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Diamond,Махагони,Нет,Нет
562,73849,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Diamond,Махагони,Нет,Нет
564,48462,Тополь,Солид Бади,NaN,Длинная,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Extreme,Махагони,Нет,Нет
565,52195,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Extreme,Махагони,Нет,Нет
573,73849,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Session,Махагони,Нет,Нет
578,48462,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Stealth,Махагони,Нет,Нет
579,82064,Тополь,Солид Бади,NaN,Длинная,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Studio,Махагони,Нет,Нет
580,82064,Тополь,Солид Бади,NaN,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Studio,Махагони,Нет,Нет


In [27]:
df['Конструкция грифа'] = df['Конструкция грифа'].fillna('Закреплён винтами')

In [28]:
df[df['Мензура'].isna()]

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Гриф,Накладка грифа,Количество ладов,Число струн,...,Страна производства,Звукосниматель грифа,Бриджевый звукосниматель,Встроенная электроника,Средний звукосниматель,Бренд,Модель,Материал корпуса,Материал корпуса_2,Материал корпуса_3
105,20833,Сполтэд Мэйпл,Солид Бади,Закреплён винтами,NaN,"34"" (864 мм)",Клён (мэйпл),Новозеландская сосна,22,четырёхструнная,...,КНР,Ибанез Стандарт Р,Ибанез Стандарт J,Фэт II ИКью (эквалайзер),NaN,Ibanez,Gio,Тополь,Нет,Нет
114,126866,Фигурный структурный клён,Солид Бади,Закреплён винтами,NaN,"34"" (864 мм)",Пятислойный панга панга / пёрпл харт,Панга панга,24,пятиструнная,...,Индонезия,Нордстрэнд Биг Сингл Вуд Каверед (пассивный),Нордстрэнд Биг Сингл Вуд Каверед (пассивный),Трёхполосный эквалайзер,NaN,Ibanez,Soundgear,Ясень,Нет,Нет
159,78910,Тополь,Солид Бади,Сплошной с декой,NaN,"34"" (864 мм)",Пятислойный клён/бубинга,Панга панга,Без ладов,Шестиструнная,...,Индонезия,Бартолини MK-1,Бартолини MK-1,Двухполосный эквалайзер,NaN,Ibanez,Bass,Махагони,Нет,Нет
275,101553,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Sandberg,Classic,Махагони,Нет,Нет
315,55929,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Omen,Махагони,Нет,Нет
561,89530,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Diamond,Махагони,Нет,Нет
562,73849,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,четырёхструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Diamond,Махагони,Нет,Нет
563,126866,Тополь,Солид Бади,Закреплён винтами,NaN,"34"" / 863 мм","3-слойный из древесины клёна, усиленный стержн...",Чёрное дерево (Эбони),22,NaN,...,Южная Корея,NaN,NaN,Двухполосный эквалайзер,NaN,Schecter,dUg,Махагони,Нет,Нет
565,52195,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Extreme,Махагони,Нет,Нет
573,73849,Тополь,Солид Бади,Закреплён винтами,NaN,NaN,NaN,NaN,NaN,пятиструнная,...,NaN,NaN,NaN,NaN,NaN,Schecter,Session,Махагони,Нет,Нет


In [29]:
df['Мензура'].value_counts()

Длинная         421
Короткая         46
Сверхдлинная     46
Малтискейл       43
Средняя          30
Name: Мензура, dtype: int64

 Признак "Мензура" фактически классифицирует значения в столбце "Длина мензуры". Если посмотреть на пропущенные значения в признаке "Мензура", то получится, что там, где длина мензуры указана "34" (864 мм)" - значение мензуры должно быть "Длинная", а где у длины мензуры пропущенные значения - "Сверхдлинная" (у этих бас-гитар "Schecter" длина мендуры - 35,5 дюймов)

In [30]:
df['Мензура'] = df['Мензура'].fillna('Длинная')

In [31]:
df['Длина мензуры'].value_counts()

34" (864 мм)                                                             373
35" (889 мм)                                                              49
34" / 863 мм                                                              38
Мультимензура "Малтискэйл" (веерные лады) 34 - 37" (864 - 940 мм)         22
30,75" (781 мм)                                                           16
30" (762 мм)                                                              16
30,5" (777 мм)                                                            12
33,25" (845 мм)                                                            8
28,63" (727 мм)                                                            7
32" (813 мм)                                                               6
Мультимензура "Малтискэйл" (веерные лады) 34 - 36,25"  (864 - 921 мм)      6
30,3" (770 мм)                                                             6
Мультимензура "Малтискэйл" (веерные лады) 34 - 35,5" (864 - 902 мм)        5

In [32]:
df['Длина мензуры'] = fill('Длина мензуры', 'Бренд')

In [33]:
for a in list(df['Длина мензуры'].unique()):
    df['Длина мензуры'] = df['Длина мензуры'].replace(a, a.split('"')[0])
df['Длина мензуры'] = df['Длина мензуры'].replace('Мультимензура ', 37)

In [34]:
df['Длина мензуры'].unique()

array(['30,5', '34', '30,3', '30,75', 37, '35', '30', '32', '33', '28,63',
       '28.6', '34,5', '33,25', '23,3'], dtype=object)

In [35]:
df['Гриф'] = fill('Гриф', 'Бренд')

In [36]:
neck_values = list(df['Гриф'].unique())

In [37]:
neck_values

['Трёхслойный клён/бук',
 'Пятислойный клён/орех',
 'Клён (мэйпл)',
 'Трёхслойный клён/махагони',
 'Пятислойный клён',
 'Махагони',
 'Пятислойный из древесины клёна, подвергнутой спец. термической обработке / ореха, дополнительно усилен графитом',
 'Девятислойный из древесины панга панга / ореха, дополнительно усилен графитом',
 'Пятислойный из древесины панга панга / пёрплхарт (амарант - ядровая древесина фиолетового оттенка) с титановыми стержнями KTS',
 'Пятислойный панга панга / пёрпл харт',
 '15-слойный панга панга / клён / падук / пёрпл харт / орех с титановой системой усиления KTS',
 'Пятислойный из древесины ятобы / ореха',
 '5-слойный из древесины махагони / ореха',
 'Пятислойный махагони/клён',
 'Пятислойный из древесины клёна, подвергнутой спец. термической обработке / пёрплхарт (амарант - ядровая древесина фиолетового оттенка) с титановыми стержнями KTS',
 'Девятислойный из древесины клёна / панга панга / ореха',
 'Обожжённый "жареный" мэйпл (клён)',
 'Цельный клён',
 'Пяти

Во многих значениях столбца "Гриф" первым словом идет количество слоев грифа - выделим это в отдельный столбец - для начала уберем из списка уникальным значений столбца "Гриф" те значения, в которых не указан слой, а затем переведем количество слоев в численные значения.

In [38]:
neck_values.remove('Клён (мэйпл)')
neck_values.remove('Махагони')
neck_values.remove('Цельный клён')
neck_values.remove('Обожжённый "жареный" мэйпл (клён)')
neck_values.remove('Выполненный из специально термически обработанной  древесины клёна, т.н. "Роустэд Мэйпл"')
neck_values.remove('Подвергнутый спец. термической обработке т.н. "жареный" клён ("Роустэд Мэйпл")')
neck_values.remove('Волокнистый клён')
neck_values.remove('Из древесины клёна, дополнительно усилен карбоном')

In [39]:
df['Кол-во слоев'] = df['Гриф']

In [40]:
for i in neck_values:
    df['Кол-во слоев'] = df['Кол-во слоев'].replace(i, i.split()[0])

In [41]:
df['Кол-во слоев'].unique()

array(['Трёхслойный', 'Пятислойный', 'Клён (мэйпл)', 'Махагони',
       'Девятислойный', '15-слойный', '5-слойный',
       'Обожжённый "жареный" мэйпл (клён)', 'Цельный клён', 'Семислойный',
       'Пятислойный,',
       'Из древесины клёна, дополнительно усилен карбоном', '3-слойный,',
       'Выполненный из специально термически обработанной  древесины клёна, т.н. "Роустэд Мэйпл"',
       '3-слойный',
       'Подвергнутый спец. термической обработке т.н. "жареный" клён ("Роустэд Мэйпл")',
       'Цельный', 'Волокнистый клён', 'Клён/орех'], dtype=object)

In [42]:
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Трёхслойный', 3)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Девятислойный', 9)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Клён (мэйпл)', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Махагони', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Пятислойный', 5)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('3-слойный', 3)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('15-слойный', 15)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Цельный клён', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Семислойный', 7)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Обожжённый "жареный" мэйпл (клён)', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('3-слойный,', 3)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Выполненный из специально термически обработанной  древесины клёна, т.н. "Роустэд Мэйпл"', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Подвергнутый спец. термической обработке т.н. "жареный" клён ("Роустэд Мэйпл")', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('5-слойный,', 5)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('5-слойный', 5)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Пятислойный,', 5)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Цельный', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Волокнистый клён', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Из древесины клёна, дополнительно усилен карбоном', 1)
df['Кол-во слоев'] = df['Кол-во слоев'].replace('Клён/орех', 2)

In [43]:
df['Кол-во слоев'].unique()

array([ 3,  5,  1,  9, 15,  7,  2])

In [44]:
neck_values = list(df['Гриф'].unique())

Для того, чтобы найти уникальные значения пород деревьев, встречающихся в столбце "Гриф" найдем леммы и затем удалим лишние леммы, оставив только породы деревьев.

In [45]:
bag = []
m = Mystem()
for i in range(len(neck_values)):
    lemmas = m.lemmatize(neck_values[i])
    for j in lemmas:
        bag.append(j)

In [46]:
pd.DataFrame(pd.Series(bag).value_counts()).index

Index([' ', '\n', 'клен', ' / ', 'древесина', 'из', 'пятислойный', ', ', '/',
       'орех', '.', 'панга', ' (', 'панг', 'мэйпл', '-', 'слойный', 'махагони',
       'бубинг', 'оттенок', 'фиолетовый', ' - ', 'амарант', 'перплхарт',
       'девятислойный', 'подвергать', 'графит', 'усильный', 'дополнительно',
       'обработка', 'термический', 'спец', 'ядровый', 'венг', 'KTS',
       'трехслойный', 'стержень', 'с', ')\n', '3', 'титановый', 'ятоба',
       'семислойный', 'усилять', 'т', 'н', 'падук', ') ', 'роустэд', 'карбон',
       '. ', '"', '  ', 'обрабатывать', 'термически', 'специально', '"\n',
       'выполнять', 'перпл', '" ', 'жареный', ' "', 'цельный', 'харт', '), ',
       'бук', 'ясень', 'палисандр', '")\n', ' ("', '15', 'ньято', 'система',
       'усиление', '5', 'обжигать', 'волокнистый'],
      dtype='object')

In [47]:
neck_wood = ['клен', 'орех', 'панг', 'панга','мэйпл', 'амарант', 'перплхарт', 'титановый', 'бубинг', 
             'венг', 'махагони', 'карбон', 'падук', 'роустэд', 'графит', 'ятоба', 'харт', 'жареный', 'перпл', 
             'ясень', 'палисандр', 'ньято', 'бук']

Напишем функцию, которая будет лемматизировать значения в каждой ячейске столбца "Гриф" и убирать леммы, которых нет в "neck_wood". Оставшиеся значения будут перечислены через запятую.

In [48]:
def neck_wood_finder(text):
    lemmas_neck = m.lemmatize(text)
    lemmas_copy = lemmas_neck.copy()
    for i in lemmas_neck:
        if i not in neck_wood:
            lemmas_copy.remove(i)
    lemmas_str = ', '.join(lemmas_copy)
    return(lemmas_str)

In [49]:
df['Гриф'] = df['Гриф'].apply(neck_wood_finder)

In [50]:
df['Гриф'].unique()

array(['клен, бук', 'клен, орех', 'клен, мэйпл', 'клен, махагони', 'клен',
       'махагони', 'клен, орех, графит', 'панга, панга, орех, графит',
       'панга, панга, перплхарт, амарант, титановый',
       'панга, панг, перпл, харт',
       'панг, панг, клен, падук, перпл, харт, орех, титановый',
       'ятоба, орех', 'махагони, орех', 'махагони, клен',
       'клен, перплхарт, амарант, титановый', 'клен, панг, панг, орех',
       'жареный, мэйпл, клен', 'венг, бубинг', 'клен, ятоба',
       'панга, панга, перплхарт, амарант, графит', 'венг, орех',
       'клен, карбон', 'клен, роустэд, мэйпл', 'клен, падук',
       'клен, венг', 'клен, бубинг', 'клен, ньято', 'ятоба, бубинг',
       'клен, перплхарт, амарант', 'клен, бубинг, панг, панг',
       'жареный, клен, роустэд, мэйпл', 'клен, палисандр', 'клен, графит',
       'венг, бубинг, клен', 'клен, ясень', 'клен, орех, падук, карбон'],
      dtype=object)

In [51]:
text = 'панга, панг, перпл, харт'

In [52]:
items_neck_materials = [
 ['панга, панга', 'панга панга'],
 ['мэйпл', 'клен'],
 ['перпл, харт', 'перплхарт'],
 ['панг, панг', 'панга панга'],
 ['титановый', 'титан'],
 ['жареный, мэйпл, клен', 'жареный клен'],
 ['роустэд, мэйпл', 'жареный клен'],
 ['бубинга', 'бубинга'],
 ['венг', 'венге']
]

In [53]:
def items_replacer(text):
    return(text.replace(items_neck_materials[i][0], items_neck_materials[i][1]))

In [54]:
for i in range(len(items_neck_materials)):
    df['Гриф'] = df['Гриф'].apply(items_replacer)

In [55]:
df['Гриф'].unique()

array(['клен, бук', 'клен, орех', 'клен, клен', 'клен, махагони', 'клен',
       'махагони', 'клен, орех, графит', 'панга панга, орех, графит',
       'панга панга, перплхарт, амарант, титан', 'панга, панг, перплхарт',
       'панга панга, клен, падук, перплхарт, орех, титан', 'ятоба, орех',
       'махагони, орех', 'махагони, клен',
       'клен, перплхарт, амарант, титан', 'клен, панга панга, орех',
       'жареный, клен, клен', 'венге, бубинг', 'клен, ятоба',
       'панга панга, перплхарт, амарант, графит', 'венге, орех',
       'клен, карбон', 'клен, роустэд, клен', 'клен, падук',
       'клен, венге', 'клен, бубинг', 'клен, ньято', 'ятоба, бубинг',
       'клен, перплхарт, амарант', 'клен, бубинг, панга панга',
       'жареный, клен, роустэд, клен', 'клен, палисандр', 'клен, графит',
       'венге, бубинг, клен', 'клен, ясень', 'клен, орех, падук, карбон'],
      dtype=object)

Каждый мателиал выведем в отдельный столбец.

In [56]:
df_neck = df['Гриф'].str.split(', ', expand = True)

In [57]:
df_neck = df_neck.fillna('Нет')

In [58]:
df_neck.columns = ['Гриф_1', 'Гриф_2', 'Гриф_3', 'Гриф_4', 'Гриф_5', 'Гриф_6']

In [59]:
df = pd.concat([df, df_neck], axis = 1)

In [60]:
df = df.drop('Гриф', axis = 1)

In [61]:
df

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,...,Материал корпуса,Материал корпуса_2,Материал корпуса_3,Кол-во слоев,Гриф_1,Гриф_2,Гриф_3,Гриф_4,Гриф_5,Гриф_6
0,25388,Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5",Палисандр (розовое дерево),22,четырёхструнная,Хамбакер,...,Клён,Нет,Нет,3,клен,бук,Нет,Нет,Нет,Нет
1,24940,Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5",Палисандр (розовое дерево),22,четырёхструнная,Соупбар,...,Клён,Нет,Нет,3,клен,бук,Нет,Нет,Нет,Нет
2,50432,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,34,Палисандр (розовое дерево),24,пятиструнная,Хамбакер,...,Ясень,Нет,Нет,5,клен,орех,Нет,Нет,Нет,Нет
3,18593,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Ятоба,22,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,1,клен,клен,Нет,Нет,Нет,Нет
4,41742,Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3",Индийский лорел (терминалия крылатая),22,четырёхструнная,Хамбакер,...,Клён,Нет,Нет,3,клен,махагони,Нет,Нет,Нет,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,88037,Тополь,Солид Бади,Закреплён винтами,Сверхдлинная,35,Чёрное дерево (Эбони),24,пятиструнная,1 x P-Стайл + 1 x J-Стайл,...,Ясень,Нет,Нет,3,клен,роустэд,клен,Нет,Нет,Нет
596,30541,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,3,клен,Нет,Нет,Нет,Нет,Нет
597,32781,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Амарант,24,четырёхструнная,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,3,клен,Нет,Нет,Нет,Нет,Нет
598,51823,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Клён (мэйпл),21,четырёхструнная,Форма ММ,...,Махагони,Нет,Нет,1,клен,клен,Нет,Нет,Нет,Нет


In [62]:
df['Накладка грифа'].value_counts()

Палисандр (розовое дерево)                                                                    134
Клён (мэйпл)                                                                                  102
Чёрное дерево (Эбони)                                                                          62
Ятоба                                                                                          56
Розовое дерево/палисандр                                                                       39
Панга панга                                                                                    30
Пау Ферро                                                                                      25
Марадо (Пау Ферро)                                                                             16
Подвергнутый спец. термической обработке, т.н. "жареный" мэйпл (клён)                          13
Амарант                                                                                        13
Многослойный шпон, о

Приведем накладки грифа к общему виду.

In [63]:
df['Накладка грифа'] = df['Накладка грифа'].replace('Палисандр (розовое дерево)', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Розовое дерево/палисандр', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Индийское розовое дерево', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Карибское розовое дерево', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Индастриэл Роузвуд', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Цельный клён (мэйпл)', 'Клён (мэйпл)')
df['Накладка грифа'] = df['Накладка грифа'].replace('Подвергнутый спец. термической обработке, т.н. "жареный" мэйпл (клён)', 'Обожжённый "жареный" мэйпл (клён)')
df['Накладка грифа'] = df['Накладка грифа'].replace('Подвергнутый спец. термической обработке т.н. "жареный" клён ("Роустэд Мэйпл")', 'Обожжённый "жареный" мэйпл (клён)')
df['Накладка грифа'] = df['Накладка грифа'].replace('Древесина клёна, подвергнутая специальной термической обработке', 'Обожжённый "жареный" мэйпл (клён)')
df['Накладка грифа'] = df['Накладка грифа'].replace('Карибское розовое дерево', 'Палисандр')
df['Накладка грифа'] = df['Накладка грифа'].replace('Панга панга с инкрустацией, выполненной из древесины орешины', 'Панга панга')
df['Накладка грифа'] = df['Накладка грифа'].replace('Макассар Эбони', 'Чёрное дерево (Эбони)')
df['Накладка грифа'] = df['Накладка грифа'].replace('Восточно-йндийское розовое дерево', 'Палисандр')

In [64]:
df['Накладка грифа'] = fill('Накладка грифа', 'Бренд')

In [65]:
df['Накладка грифа'].value_counts()

Палисандр                                                                                     186
Клён (мэйпл)                                                                                  105
Чёрное дерево (Эбони)                                                                          69
Ятоба                                                                                          56
Панга панга                                                                                    33
Индийский лорел (терминалия крылатая)                                                          26
Пау Ферро                                                                                      25
Обожжённый "жареный" мэйпл (клён)                                                              22
Марадо (Пау Ферро)                                                                             16
Амарант                                                                                        13
Многослойный шпон, о

In [66]:
df['Количество ладов'].value_counts()

24           240
22           193
21            86
20            60
Без ладов      2
27             1
Name: Количество ладов, dtype: int64

In [67]:
df['Количество ладов'] = df['Количество ладов'].replace('Без ладов', 0)

In [68]:
df['Количество ладов'] = fill('Количество ладов', 'Длина мензуры')

In [69]:
df['Число струн'].value_counts()

четырёхструнная    370
пятиструнная       197
Шестиструнная       24
восьмиструнная       5
Семиструнная         1
4                    1
Name: Число струн, dtype: int64

In [70]:
df['Число струн'] = df['Число струн'].replace('четырёхструнная', 4)
df['Число струн'] = df['Число струн'].replace('пятиструнная', 5)
df['Число струн'] = df['Число струн'].replace('Шестиструнная', 6)
df['Число струн'] = df['Число струн'].replace('восьмиструнная', 8)
df['Число струн'] = df['Число струн'].replace('Семиструнная', 7)
df['Число струн'] = df['Число струн'].replace('4', 4)

In [71]:
df[df['Число струн'].isna()]['Модель']

563       dUg
582    Studio
Name: Модель, dtype: object

In [72]:
df['Число струн'] = df['Число струн'].fillna(4)
df['Число струн'] = df['Число струн'].astype(int)

In [73]:
df['Число струн'].value_counts()

4    373
5    197
6     24
8      5
7      1
Name: Число струн, dtype: int64

In [74]:
df['Типы звукоснимателей'].value_counts()

Хамбакер                              161
1 x P-Стайл + 1 x J-Стайл              84
Соупбар                                53
2 х J Стайл                            42
1 x J-Стайл, 1 x MM-Стайл              42
Сингл Койл                             36
1 x P-Стайл + 1 x MM-Стайл             34
Соупбар Хамбакер                       28
Форма ММ                               24
P-Стайл                                14
MM-Стайл + пьезо                        3
Липстик                                 2
Хамбакер + пьезо                        2
Сингл Койл P-Стайл                      2
S-H  (Сингл Койл + Хамбакер)            1
Мини Хамбакер                           1
Соупбар Хамбакер + бриджевый пьезо      1
Хамбакер + Мини Хамбакер                1
Сингл Койл + пьезо                      1
Сингл Койл в стиле Рикенбекер           1
2 x P-Стайл                             1
Name: Типы звукоснимателей, dtype: int64

In [75]:
df['Типы звукоснимателей'] = df['Типы звукоснимателей'].replace('Соупбар', 'Хамбакер')
df['Типы звукоснимателей'] = df['Типы звукоснимателей'].replace('Соупбар Хамбакер', 'Хамбакер')

In [76]:
df['Типы звукоснимателей'] = fill('Типы звукоснимателей', 'Бренд')

In [77]:
df['Страна производства'].value_counts()

Индонезия      176
Германия       159
КНР             57
Южная Корея     37
США             19
Чехия           19
Япония           7
Name: Страна производства, dtype: int64

In [78]:
df['Страна производства'] = fill('Страна производства', 'Бренд')

In [79]:
df['Звукосниматель грифа'] = df['Звукосниматель грифа'].fillna('Нет')
df['Бриджевый звукосниматель'] = df['Бриджевый звукосниматель'].fillna('Нет')
df['Средний звукосниматель'] = df['Средний звукосниматель'].fillna('Нет')

In [80]:
df['Встроенная электроника'].unique()

array([nan, 'Трёхполосный эквалайзер', 'Фэт II ИКью (эквалайзер)',
       'Двухполосный эквалайзер', 'Четырёхполосный эквалайзер'],
      dtype=object)

In [81]:
df['Встроенная электроника'] = df['Встроенная электроника'].fillna('Нет')

In [82]:
df

,Цена,Дека,Конструкция,Конструкция грифа,Мензура,Длина мензуры,Накладка грифа,Количество ладов,Число струн,Типы звукоснимателей,...,Материал корпуса,Материал корпуса_2,Материал корпуса_3,Кол-во слоев,Гриф_1,Гриф_2,Гриф_3,Гриф_4,Гриф_5,Гриф_6
0,25388,Ель,Полуакустическая (Семи Холлоу Бади),Закреплён винтами,Короткая,"30,5",Палисандр,22,4,Хамбакер,...,Клён,Нет,Нет,3,клен,бук,Нет,Нет,Нет,Нет
1,24940,Ель,Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,5",Палисандр,22,4,Хамбакер,...,Клён,Нет,Нет,3,клен,бук,Нет,Нет,Нет,Нет
2,50432,"Клён, расщеплённый (неповторимый эффект)",Солид Бади,Закреплён винтами,Длинная,34,Палисандр,24,5,Хамбакер,...,Ясень,Нет,Нет,5,клен,орех,Нет,Нет,Нет,Нет
3,18593,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Ятоба,22,4,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,1,клен,клен,Нет,Нет,Нет,Нет
4,41742,Клён (мэйпл),Полуакустическая (Семи Холлоу Бади),Вклеен в корпус,Короткая,"30,3",Индийский лорел (терминалия крылатая),22,4,Хамбакер,...,Клён,Нет,Нет,3,клен,махагони,Нет,Нет,Нет,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,88037,Тополь,Солид Бади,Закреплён винтами,Сверхдлинная,35,Чёрное дерево (Эбони),24,5,1 x P-Стайл + 1 x J-Стайл,...,Ясень,Нет,Нет,3,клен,роустэд,клен,Нет,Нет,Нет
596,30541,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Амарант,24,4,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,3,клен,Нет,Нет,Нет,Нет,Нет
597,32781,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Амарант,24,4,1 x P-Стайл + 1 x J-Стайл,...,Нато,Нет,Нет,3,клен,Нет,Нет,Нет,Нет,Нет
598,51823,Тополь,Солид Бади,Закреплён винтами,Длинная,34,Клён (мэйпл),21,4,Форма ММ,...,Махагони,Нет,Нет,1,клен,клен,Нет,Нет,Нет,Нет


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Цена                      600 non-null    int64 
 1   Дека                      600 non-null    object
 2   Конструкция               600 non-null    object
 3   Конструкция грифа         600 non-null    object
 4   Мензура                   600 non-null    object
 5   Длина мензуры             600 non-null    object
 6   Накладка грифа            600 non-null    object
 7   Количество ладов          600 non-null    object
 8   Число струн               600 non-null    int64 
 9   Типы звукоснимателей      600 non-null    object
 10  Страна производства       600 non-null    object
 11  Звукосниматель грифа      600 non-null    object
 12  Бриджевый звукосниматель  600 non-null    object
 13  Встроенная электроника    600 non-null    object
 14  Средний звукосниматель    

In [84]:
df.to_csv('musik_productiv_df_processed', index = False)

## Вывод

Данные в каждом столбце приведены к общему виду и заполнены на 100% - датафрейм готов к анализу, проверке статистических гипотез и машинному обучению.